In [9]:
import pandas as pd
import numpy as np

In [10]:
Fan = []
for i in range(6):
    dataFile = "PredictData/predict_for_fan" + str(i) + ".csv"
    temp = pd.read_csv(dataFile)
    Fan.append(temp)

In [11]:
for i in range(6):
    Fan[i]["month"] = Fan[i]["time"].str.slice(5, 7).astype("int64")
    Fan[i]["day"] = Fan[i]["time"].str.slice(8, 10).astype("int64")
    Fan[i]["hour"] = Fan[i]["time"].str.slice(11, 13).astype("int64")
    Fan[i]["minute"] = Fan[i]["time"].str.slice(14, 16).astype("int64")
    Fan[i]["second"] = Fan[i]["time"].str.slice(17, 19).astype("int64")
    del Fan[i]["time"]

In [12]:
for i in range(6):
    Fan[i].loc[:, "hour"] += 8
    Fan[i].loc[Fan[i]["hour"] >= 24, "day"] += 1
    Fan[i].loc[Fan[i]["hour"] >= 24, "hour"] -= 24
    t = (Fan[i]["day"] == 32)
    Fan[i].loc[t, "month"] += 1
    Fan[i].loc[t, "day"] -= 31
    t = (Fan[i]["day"] == 31) & ((Fan[i]["month"] == 4) | (Fan[i]["month"] == 6))
    Fan[i].loc[t, "month"] += 1
    Fan[i].loc[t, "day"] -= 30
    Fan[i]["time"] = Fan[i]["hour"] * 3600 + Fan[i]["minute"] * 60 + Fan[i]["second"]
    Fan[i]["dd"] = Fan[i]["mm"] * 100 + Fan[i]["dd"]
    Fan[i]["dd"] = Fan[i]["dd"].astype("int64")
    del Fan[i]["mm"]
    del Fan[i]["hour"]
    del Fan[i]["minute"]
    del Fan[i]["second"]
    Fan[i] = Fan[i].reindex(columns = ["month", "day", "time", "speed", "ntspeed", "P", "T", "u", "v", "w", "dd"])

In [13]:
for i in range(6):
    Fan[i]["new"] = False
    Fan[i] = Fan[i].sort_values(by = ["month", "day", "time", "dd"])
    Fan[i].index = range(42978)

In [14]:
for i in range(6):
    for j in range(42977):
        t1 = Fan[i].loc[j,:]
        t2 = Fan[i].loc[j+1,:]
        if (t1["month"] == t2["month"]) and (t1["day"] == t2["day"]) and (t1["time"] == t2["time"]):
            continue
        else:
            Fan[i].loc[j, "new"] = True
    Fan[i].loc[42977,"new"] = True
    Fan[i] = Fan[i][Fan[i]["month"] != 8]

In [17]:
new = []
old = []
for i in range(6):
    s1 = Fan[i][Fan[i]["new"] == True]
    s2 = Fan[i][Fan[i]["new"] == False]
    s1.index = range(11697)
    s2.index = range(30990)
    del s1["new"]
    del s2["new"]
    new.append(s1)
    old.append(s2)

In [18]:
for i in range(6):
    dataFile = "PredictData/new_predict_for_fan" + str(i) + ".csv"
    new[i].to_csv(dataFile, index = False)
    dataFile = "PredictData/old_predict_for_fan" + str(i) + ".csv"
    old[i].to_csv(dataFile, index = False)
    dataFile = "PredictData/all_predict_for_fan" + str(i) + ".csv"
    Fan[i].to_csv(dataFile, index = False)